In [1]:
# Análisis Exploratorio de Datos - Dataset Yelp
# Filtrado de negocios en Montreal

import pandas as pd
import json

print("Iniciando análisis exploratorio del dataset de negocios de Yelp")


Iniciando análisis exploratorio del dataset de negocios de Yelp


In [2]:
# Leer el archivo JSON línea por línea y convertir a DataFrame
# El archivo contiene un objeto JSON por línea

data = []
file_path = '../data/raw/yelp_academic_dataset_business.json'

print(f"Leyendo archivo: {file_path}")

with open(file_path, 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file, 1):
        try:
            json_object = json.loads(line.strip())
            data.append(json_object)
        except json.JSONDecodeError as e:
            print(f"Error en la línea {line_number}: {e}")
        
        # Mostrar progreso cada 10000 líneas
        if line_number % 10000 == 0:
            print(f"Procesadas {line_number} líneas...")

print(f"Total de registros cargados: {len(data)}")

# Convertir a DataFrame
df_business = pd.DataFrame(data)
print(f"Dimensiones del DataFrame: {df_business.shape}")
print(f"Columnas: {list(df_business.columns)}")


Leyendo archivo: ../data/raw/yelp_academic_dataset_business.json
Procesadas 10000 líneas...
Procesadas 20000 líneas...
Procesadas 30000 líneas...
Procesadas 40000 líneas...
Procesadas 50000 líneas...
Procesadas 60000 líneas...
Procesadas 70000 líneas...
Procesadas 80000 líneas...
Procesadas 90000 líneas...
Procesadas 100000 líneas...
Procesadas 110000 líneas...
Procesadas 120000 líneas...
Procesadas 130000 líneas...
Procesadas 140000 líneas...
Procesadas 150000 líneas...
Total de registros cargados: 150346
Dimensiones del DataFrame: (150346, 14)
Columnas: ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']


In [6]:
# Explorar las ciudades únicas para entender la variación en los nombres
print("Explorando ciudades únicas...")
print(f"Total de ciudades únicas: {df_business['city'].nunique()}")
print("\nPrimeras 20 ciudades únicas:")
print(df_business['city'].value_counts().head(100))

# Buscar variaciones de Montreal
montreal_variations = df_business[df_business['city'].str.contains('Toronto', case=False, na=False)]['city'].unique()
print(f"\nVariaciones de Montreal encontradas: {montreal_variations}")




Explorando ciudades únicas...
Total de ciudades únicas: 1416

Primeras 20 ciudades únicas:
city
Philadelphia    14569
Tucson           9250
Tampa            9050
Indianapolis     7540
Nashville        6971
                ...  
Voorhees          241
Zionsville        234
Antioch           231
Hudson            228
Arnold            227
Name: count, Length: 100, dtype: int64

Variaciones de Montreal encontradas: []


In [ ]:
# Filtrar negocios en Montreal
# Usamos regex para capturar variaciones de Montreal/Montréal

montreal_businesses = df_business[df_business['city'].str.contains('Montr[eé]al', case=False, na=False, regex=True)]

print(f"Número de negocios encontrados en Montreal: {len(montreal_businesses)}")

if len(montreal_businesses) > 0:
    print(f"\nInformación básica sobre los negocios de Montreal:")
    print(f"- Total de negocios: {len(montreal_businesses)}")
    print(f"- Negocios abiertos: {montreal_businesses['is_open'].sum()}")
    print(f"- Negocios cerrados: {len(montreal_businesses) - montreal_businesses['is_open'].sum()}")
    print(f"- Puntuación promedio: {montreal_businesses['stars'].mean():.2f}")
    print(f"- Número promedio de reseñas: {montreal_businesses['review_count'].mean():.2f}")
    
    # Mostrar información detallada de los primeros negocios
    print(f"\nPrimeros 5 negocios en Montreal:")
    display_columns = ['name', 'address', 'city', 'state', 'postal_code', 'stars', 'review_count', 'is_open', 'categories']
    print(montreal_businesses[display_columns].head())
else:
    print("No se encontraron negocios en Montreal en el dataset.")


In [ ]:
# Análisis adicional si encontramos negocios en Montreal
if len(montreal_businesses) > 0:
    
    # Análisis por categorías
    print("=== ANÁLISIS POR CATEGORÍAS ===")
    
    # Expandir categorías (están separadas por comas)
    categories_list = []
    for categories in montreal_businesses['categories'].dropna():
        if categories:
            cat_split = [cat.strip() for cat in categories.split(',')]
            categories_list.extend(cat_split)
    
    if categories_list:
        categories_df = pd.Series(categories_list).value_counts()
        print(f"Top 10 categorías más comunes en Montreal:")
        print(categories_df.head(10))
    
    # Análisis por estado/provincia
    print(f"\n=== DISTRIBUCIÓN POR PROVINCIA/ESTADO ===")
    print(montreal_businesses['state'].value_counts())
    
    # Análisis de puntuaciones
    print(f"\n=== ANÁLISIS DE PUNTUACIONES ===")
    print(f"Distribución de estrellas:")
    print(montreal_businesses['stars'].value_counts().sort_index())
    
    # Guardar los resultados filtrados en un nuevo archivo CSV para análisis posterior
    output_file = '../data/montreal_businesses.csv'
    montreal_businesses.to_csv(output_file, index=False)
    print(f"\nLos negocios de Montreal han sido guardados en: {output_file}")
    
else:
    print("No hay negocios de Montreal para analizar más detalladamente.")


In [ ]:
# Análisis del Dataset de Reviews
# Estructura: {"review_id", "user_id", "business_id", "stars", "useful", "funny", "cool", "text", "date"}

print("=== ANÁLISIS ESTRUCTURA DATASET REVIEWS ===")

# Leer una muestra de reviews para análisis
reviews_sample = []
reviews_file = '../data/raw/yelp_academic_dataset_review.json'

print(f"Leyendo muestra de reviews de: {reviews_file}")

with open(reviews_file, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i < 1000:  # Leer solo las primeras 1000 reviews para análisis inicial
            try:
                review = json.loads(line.strip())
                reviews_sample.append(review)
            except json.JSONDecodeError as e:
                print(f"Error en línea {i+1}: {e}")
        else:
            break

# Convertir a DataFrame
df_reviews_sample = pd.DataFrame(reviews_sample)
print(f"Muestra de reviews cargada: {len(df_reviews_sample)} registros")
print(f"Columnas del dataset de reviews: {list(df_reviews_sample.columns)}")
print(f"\nTipos de datos:")
print(df_reviews_sample.dtypes)


In [ ]:
# Análisis exploratorio de la muestra de reviews
print("=== ANÁLISIS EXPLORATORIO REVIEWS ===")

# Estadísticas básicas
print(f"Distribución de estrellas en la muestra:")
print(df_reviews_sample['stars'].value_counts().sort_index())

print(f"\nEstadísticas de utilidad, diversión y coolness:")
print(df_reviews_sample[['useful', 'funny', 'cool']].describe())

# Análisis de longitud del texto
df_reviews_sample['text_length'] = df_reviews_sample['text'].str.len()
print(f"\nEstadísticas de longitud del texto:")
print(df_reviews_sample['text_length'].describe())

# Mostrar ejemplos de reviews por puntuación
print(f"\n=== EJEMPLOS DE REVIEWS POR PUNTUACIÓN ===")
for stars in [1, 3, 5]:
    sample_review = df_reviews_sample[df_reviews_sample['stars'] == stars]['text'].iloc[0]
    print(f"\n--- REVIEW {stars} ESTRELLAS ---")
    print(f"Texto: {sample_review[:200]}...")
    print(f"Longitud: {len(sample_review)} caracteres")
